
# Databricks Demo Notebook
This notebook demonstrates how to access IDB data hosted on databricks through the `msk_cdm` python package (via Databricks Python SDK)


## Setup
After creating your conda environment including the `msk_cdm` package (See [README](https://github.com/clinical-data-mining/msk_cdm/blob/main/README.md)), the user will need to collect (3) connection details from the Databricks UI, and create an API token. These are the items to collect:
- API Token
- Server Hostname
- HTTP Path
- Host URL (Typically a https extension of Server Hostname)

### Finding the connection details
- [Finding the connection details to Databricks](https://docs.databricks.com/en/integrations/compute-details.html)
- [Creating a token](https://docs.databricks.com/en/dev-tools/auth/pat.html)

## The 'msk_cdm' Databricks module
The Databricks module can be instantiated by this import function:
```
from msk_cdm.databricks import DatabricksAPI

obj_dbk = DatabricksAPI(    
    TOKEN=token,
    URL=url,
    HTTP_PATH=http_path,
    HOSTNAME=hostname
)
```


### Creating a Databricks environment file 
If the user would like to store connection details in a file and instantiating through the file instead of individual connection details, the user can create a file with this template:

`<pathname>/env_databricks.txt`:
```
TOKEN=<YOUR_TOKEN>
URL=<THE_URL>
HOSTNAME=<THE_HOSTNAME>
HTTP_PATH=<THE_HTTP_PATH>
```

Then, instantiate in Python with
```
f = '<pathname>/env_databricks.txt'
obj_dbk = DatabricksAPI(fname_databricks_env=f)
```

---

In [ ]:
from msk_cdm.databricks import DatabricksAPI

In [ ]:
token = '<YOUR_TOKEN>'
url = '<THE_URL'
hostname = '<THE_HOSTNAME>'
http_path = '<THE_HTTP_PATH>'

## Instantiate the Databricks Object

### Try with the individual connection details

In [0]:
obj_dbk = DatabricksAPI(
    token=token,
    http_path=http_path,
    hostname=hostname
)

### Attempt the same with the environment file
 

In [ ]:
fname_env = 'databricks_env.txt'
obj_dbk = DatabricksAPI(fname_databricks_env=fname_env)

## Define some Spark SQL to make a Query 

In [ ]:
sql = f"""
USE CATALOG mode_clinical_test;

SELECT DISTINCT
    t1.pt_mrn AS MRN,
    t1.pt_birth_dte AS PT_BIRTH_DTE,
    t1.pt_death_dte AS PT_DEATH_DTE,
    t1.pt_mrn_create_dte AS MRN_CREATE_DTE,
    t1.pt_sex_desc AS GENDER,
    t1.pt_marital_sts_desc AS MARITAL_STATUS,
    t1.pt_religion_desc AS RELIGION,
    t1.pt_race_desc AS RACE,
    t4.core_ethnicity_desc AS ETHNICITY,
    t6.pla_last_contact_dte AS PLA_LAST_CONTACT_DTE,
    t6.pla_last_actv_dte AS PLA_LAST_ACTV_DTE,
    t6.pla_last_adm_dte AS PLA_LAST_ADM_DTE,
    t6.pla_last_dsch_dte AS PLA_LAST_DSCH_DTE,
    t6.pla_last_appt_dte AS PLA_LAST_APPT_DTE,
    t6.pla_last_drvst_dte AS PLA_LAST_DRVST_DTE,
    t6.pla_last_tx_dte AS PLA_LAST_TX_DTE,
    t6.pla_last_chemo_dte AS PLA_LAST_CHEMO_DTE,
    t6.pla_last_surg_dte AS PLA_LAST_SURG_DTE,
    t6.pla_last_rt_dte AS PLA_LAST_RT_DTE,
    CASE
        WHEN DATEDIFF(COALESCE(t1.pt_death_dte, CURRENT_DATE()), t1.pt_birth_dte) / 365.25 < 90 THEN
            CAST(DATEDIFF(COALESCE(t1.pt_death_dte, CURRENT_DATE()), t1.pt_birth_dte) / 365.25 AS INT)
        ELSE
            89
        END AS CURRENT_AGE_DEID
FROM
    dcmspt.patient_demographics_v t1
        INNER JOIN dcmspt.patient_latest_activity t6 ON t1.pt_pt_deidentification_id = t6.pla_pt_deidentification_id
        LEFT JOIN dcmspt.patient_core_demographics t4 ON t1.pt_pt_deidentification_id = t4.core_pt_deidentification_id

"""

In [ ]:
print(sql)

## Run the Query!

In [ ]:
df = obj_dbk.query_from_sql(sql=sql)

In [ ]:
df.head();

## Close connection with Databricks
When complete with queries. We may get emails from Databricks maintainers if connections aren't closed.

In [ ]:
obj_dbk.close_connection()